<a href="https://colab.research.google.com/github/IjeomaOdoko/Federal-Debt-GDP-and-Oil-Prices/blob/master/Federal_Debt%2C_Population_and_GDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All datasets are going to use Q1 or March as Fiscal Year End with the following exceptions: 

* Data not provided for 2009 March Federal Debt, Sep 2009 value assumed for that year. 

Datasets used are as follows: 

Statistics Canada. [Table 36-10-0104-01 Gross domestic product, expenditure-based, Canada, quarterly (x 1,000,000)](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3610010401&pickMembers%5B0%5D=2.2&pickMembers%5B1%5D=3.1&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=1961&cubeTimeFrame.endMonth=04&cubeTimeFrame.endYear=2020&referencePeriods=19610101%2C20200401)

Statistics Canada. [Table 17-10-0009-01 Population estimates, quarterly](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1710000901&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=1960&cubeTimeFrame.endMonth=04&cubeTimeFrame.endYear=2020&referencePeriods=19600101%2C20200401)


Statistics Canada. [Table 10-10-0048-01 Federal government debt, for fiscal year ending March 31](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1010004801&cubeTimeFrame.startYear=1960&cubeTimeFrame.endYear=2008&referencePeriods=19600101%2C20080101)

Statistics Canada.[Table 10-10-0002-01 Central government debt (x 1,000,000)](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1010000201&cubeTimeFrame.startMonth=01&cubeTimeFrame.startYear=2009&cubeTimeFrame.endMonth=05&cubeTimeFrame.endYear=2020&referencePeriods=20090101%2C20200501)






# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

from google.colab import files


# Load Datasets

In [ ]:
uploaded = files.upload()

Saving 3610010401-eng.csv to 3610010401-eng.csv


In [ ]:
uploaded_1 = files.upload()

Saving 1710000901-eng.csv to 1710000901-eng.csv


In [ ]:
uploaded_2 = files.upload()

Saving 1010004801-eng.csv to 1010004801-eng.csv


In [ ]:
uploaded_3 = files.upload()

Saving 10100002.csv to 10100002.csv


In [ ]:
import io

canadian_gdp= pd.read_csv(io.StringIO(uploaded['3610010401-eng.csv'].decode('utf-8')), header=7, index_col='Estimates').dropna(how='any')

canadian_pop_estimates= pd.read_csv(io.StringIO(uploaded_1['1710000901-eng.csv'].decode('utf-8')), header=4, index_col='Geography').dropna(how='any')

canadian_fed_debt_archived= pd.read_csv(io.StringIO(uploaded_2['1010004801-eng.csv'].decode('utf-8')), header=4, index_col='Government debt').dropna(how='any')

canadian_fed_debt_new=pd.read_csv(io.StringIO(uploaded_3['10100002.csv'].decode('utf-8')), header=0,  parse_dates=['REF_DATE'])



# Transform GDP Dataset 

In [ ]:
canadian_gdp.head()

,Q1 1961,Q2 1961,Q3 1961,Q4 1961,Q1 1962,Q2 1962,Q3 1962,Q4 1962,Q1 1963,Q2 1963,Q3 1963,Q4 1963,Q1 1964,Q2 1964,Q3 1964,Q4 1964,Q1 1965,Q2 1965,Q3 1965,Q4 1965,Q1 1966,Q2 1966,Q3 1966,Q4 1966,Q1 1967,Q2 1967,Q3 1967,Q4 1967,Q1 1968,Q2 1968,Q3 1968,Q4 1968,Q1 1969,Q2 1969,Q3 1969,Q4 1969,Q1 1970,Q2 1970,Q3 1970,Q4 1970,...,Q3 2010,Q4 2010,Q1 2011,Q2 2011,Q3 2011,Q4 2011,Q1 2012,Q2 2012,Q3 2012,Q4 2012,Q1 2013,Q2 2013,Q3 2013,Q4 2013,Q1 2014,Q2 2014,Q3 2014,Q4 2014,Q1 2015,Q2 2015,Q3 2015,Q4 2015,Q1 2016,Q2 2016,Q3 2016,Q4 2016,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Q1 2018,Q2 2018,Q3 2018,Q4 2018,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Q1 2020,Q2 2020
Estimates,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Final consumption expenditure,"32,125","32,732","33,712","33,923","34,748","35,352","35,371","36,285","36,739","37,570","38,163","39,036","39,991","40,230","41,097","42,034","42,543","43,867","44,927","46,267","47,475","48,578","50,042","51,073","51,902","53,929","54,640","56,153","57,484","58,523","60,550","62,319","63,784","64,827","66,458","68,135","69,258","70,874","71,930","73,174",...,"1,312,704","1,332,504","1,345,104","1,360,452","1,374,388","1,387,620","1,395,452","1,400,720","1,413,340","1,423,992","1,439,696","1,449,384","1,465,740","1,478,632","1,494,408","1,511,832","1,525,620","1,537,644","1,545,556","1,555,784","1,576,564","1,586,956","1,593,000","1,603,968","1,615,308","1,631,540","1,655,236","1,677,300","1,690,740","1,714,976","1,731,136","1,744,728","1,764,152","1,777,288","1,793,676","1,810,884","1,826,212","1,845,248","1,808,160","1,624,480"
Household final consumption expenditure,"25,196","25,684","26,328","26,520","27,204","27,516","27,540","28,240","28,532","29,252","29,604","30,288","30,888","31,120","31,888","32,420","32,756","33,860","34,556","35,608","36,264","36,880","37,824","38,316","38,928","40,168","40,596","41,816","42,836","43,436","44,664","45,752","47,088","47,596","48,508","49,640","49,716","50,348","51,440","52,124",...,"930,732","945,200","952,556","963,132","970,660","981,988","989,668","993,188","1,000,460","1,008,896","1,021,024","1,032,568","1,044,832","1,053,920","1,065,680","1,082,060","1,093,188","1,104,084","1,105,264","1,114,204","1,129,568","1,139,280","1,141,184","1,147,040","1,156,132","1,170,316","1,187,036","1,202,500","1,212,884","1,231,332","1,241,284","1,248,716","1,262,964","1,268,344","1,279,836","1,291,256","1,301,216","1,313,352","1,274,724","1,100,516"
Goods,"15,260","15,664","16,064","16,060","16,768","16,896","16,844","17,380","17,536","17,960","18,100","18,560","18,944","19,004","19,472","19,676","19,924","20,664","21,112","21,768","22,168","22,364","22,972","23,080","23,436","24,448","24,648","25,032","25,516","25,808","26,476","27,048","27,932","28,116","28,500","29,220","29,048","29,168","29,924","30,540",...,"417,924","424,940","426,912","432,016","434,864","439,972","444,196","441,824","444,528","447,984","452,848","457,256","463,880","468,016","472,496","480,732","484,800","487,280","483,656","488,232","495,888","500,012","500,092","503,020","503,492","512,460","522,708","530,364","532,724","542,568","545,932","550,320","555,532","553,308","557,496","561,908","564,516","567,776","553,384","498,720"
Durable goods,"2,828","2,984","3,212","3,160","3,404","3,408","3,312","3,548","3,564","3,772","3,680","4,024","4,136","4,096","4,256","4,144","4,380","4,688","4,768","4,892","5,084","4,756","5,160","5,276","5,188","5,656","5,588","5,548","5,788","5,928","6,140","6,344","6,680","6,532","6,556","6,776","6,324","6,392","6,660","6,460",...,"117,064","118,588","116,968","117,480","117,100","120,100","123,160","120,724","120,644","121,516","124,284","127,604","128,172","127,800","129,568","133,724","136,556","137,620","135,892","138,616","142,448","145,624","147,652","147,436","147,748","151,780","157,976","161,360","161,328","164,424","164,764","164,952","165,352","164,816","167,296","166,924","167,756","167,768","153,288","128,244"
Semi-durable g

In [ ]:
   canadian_gdp.dtypes

Q1 1961    object
Q2 1961    object
Q3 1961    object
Q4 1961    object
Q1 1962    object
            ...  
Q2 2019    object
Q3 2019    object
Q4 2019    object
Q1 2020    object
Q2 2020    object
Length: 238, dtype: object

In [ ]:
GDP = canadian_gdp.loc['Gross domestic product at market prices']

GDP

Q1 1961       40,672
Q2 1961       41,568
Q3 1961       42,680
Q4 1961       43,476
Q1 1962       44,696
             ...    
Q2 2019    2,305,784
Q3 2019    2,312,064
Q4 2019    2,335,700
Q1 2020    2,275,320
Q2 2020    1,989,784
Name: Gross domestic product at market prices, Length: 238, dtype: object

In [ ]:
# check class/type
print(type(GDP))

<class 'pandas.core.series.Series'>


In [ ]:
# remove commas to be able to change to numeric later
GDP = GDP.str.replace(',','', regex=True)

In [ ]:
# convert series to dataframe

GDP_year= pd.DataFrame(GDP, columns=['Gross domestic product at market prices'])
GDP_year.head()

,Gross domestic product at market prices
Q1 1961,40672
Q2 1961,41568
Q3 1961,42680
Q4 1961,43476
Q1 1962,44696


In [ ]:
GDP_year.reset_index(inplace=True)

GDP_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 2 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   index                                    238 non-null    object
 1   Gross domestic product at market prices  238 non-null    object
dtypes: object(2)
memory usage: 3.8+ KB


In [ ]:
# change dtypes to integer and string

GDP_year['Gross domestic product at market prices'] = GDP_year['Gross domestic product at market prices'].astype('int64')
GDP_year['index'] = GDP_year['index'].astype('string')

GDP_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 2 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   index                                    238 non-null    string
 1   Gross domestic product at market prices  238 non-null    int64 
dtypes: int64(1), string(1)
memory usage: 3.8 KB


In [ ]:
## add columns for Quarter and Year by splitting Quarter_Year Column

splitframes= GDP_year['index'].str.split(' ', n=1, expand=True)

GDP_year['Quarter'] = splitframes[0]
GDP_year['Year'] = splitframes[1].astype('int64')

In [ ]:
# drop all rows where Quarter is not Q1

GDP_year_Q1 = GDP_year.loc[GDP_year['Quarter'] == 'Q1']

GDP_year_Q1.head()

,index,Gross domestic product at market prices,Quarter,Year
0,Q1 1961,40672,Q1,1961
4,Q1 1962,44696,Q1,1962
8,Q1 1963,47668,Q1,1963
12,Q1 1964,52652,Q1,1964
16,Q1 1965,57364,Q1,1965


In [ ]:
# drop quarter_year, and quarter columns
GDP_year_df = GDP_year_Q1.drop(columns=['index','Quarter'], inplace=False).set_index('Year')

GDP_year_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1961 to 2020
Data columns (total 1 columns):
 #   Column                                   Non-Null Count  Dtype
---  ------                                   --------------  -----
 0   Gross domestic product at market prices  60 non-null     int64
dtypes: int64(1)
memory usage: 960.0 bytes


In [ ]:
GDP_year_df.rename(columns={'Gross domestic product at market prices': "Gross domestic product at market prices (million dollars)"}, inplace=True)
GDP_year_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1961 to 2020
Data columns (total 1 columns):
 #   Column                                                     Non-Null Count  Dtype
---  ------                                                     --------------  -----
 0   Gross domestic product at market prices (million dollars)  60 non-null     int64
dtypes: int64(1)
memory usage: 960.0 bytes


# Transform Pop Estimates Dataset 

In [ ]:
canadian_pop_estimates.head()

,Q1 1960,Q2 1960,Q3 1960,Q4 1960,Q1 1961,Q2 1961,Q3 1961,Q4 1961,Q1 1962,Q2 1962,Q3 1962,Q4 1962,Q1 1963,Q2 1963,Q3 1963,Q4 1963,Q1 1964,Q2 1964,Q3 1964,Q4 1964,Q1 1965,Q2 1965,Q3 1965,Q4 1965,Q1 1966,Q2 1966,Q3 1966,Q4 1966,Q1 1967,Q2 1967,Q3 1967,Q4 1967,Q1 1968,Q2 1968,Q3 1968,Q4 1968,Q1 1969,Q2 1969,Q3 1969,Q4 1969,...,Q3 2010,Q4 2010,Q1 2011,Q2 2011,Q3 2011,Q4 2011,Q1 2012,Q2 2012,Q3 2012,Q4 2012,Q1 2013,Q2 2013,Q3 2013,Q4 2013,Q1 2014,Q2 2014,Q3 2014,Q4 2014,Q1 2015,Q2 2015,Q3 2015,Q4 2015,Q1 2016,Q2 2016,Q3 2016,Q4 2016,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Q1 2018,Q2 2018,Q3 2018,Q4 2018,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Q1 2020,Q2 2020
Geography,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Canada,"17,710,000","17,793,000","17,909,000","18,009,000","18,092,000","18,172,000","18,271,000","18,363,000","18,442,000","18,519,000","18,614,000","18,708,000","18,787,000","18,864,000","18,964,000","19,061,000","19,142,000","19,222,000","19,325,000","19,420,000","19,501,000","19,578,000","19,678,000","19,777,000","19,857,000","19,939,000","20,048,000","20,146,000","20,228,000","20,306,000","20,412,000","20,509,000","20,581,000","20,644,000","20,729,000","20,814,000","20,888,000","20,950,000","21,028,000","21,111,000",...,"34,004,889","34,131,683","34,166,099","34,230,378","34,339,328","34,457,998","34,516,352","34,592,779","34,714,222","34,836,008","34,883,119","34,958,216","35,082,954","35,211,866","35,249,639","35,323,533","35,437,435","35,559,047","35,575,187","35,611,271","35,702,908","35,822,894","35,871,136","35,970,303","36,109,487","36,258,404","36,313,430","36,396,785","36,543,321","36,714,716","36,788,832","36,889,543","37,057,765","37,238,906","37,311,904","37,408,205","37,589,262","37,797,496","37,894,799","37,971,020"
Newfoundland and Labrador,"445,000","447,000","449,000","452,000","454,000","456,000","459,000","462,000","464,000","466,000","469,000","471,000","473,000","474,000","477,000","479,000","480,000","482,000","484,000","485,000","485,000","486,000","488,000","489,000","490,000","492,000","494,000","496,000","496,000","498,000","500,000","502,000","503,000","505,000","507,000","509,000","511,000","513,000","515,000","516,000",...,"522,009","522,865","523,726","524,239","524,999","525,936","526,215","525,655","526,345","527,180","527,248","527,313","527,114","528,117","528,242","527,498","528,159","528,461","528,361","528,063","528,117","528,696","528,800","529,063","529,426","529,943","529,438","528,650","528,356","528,334","527,595","526,493","525,604","525,258","524,053","522,818","521,542","521,922","521,365","520,437"
Prince Edward Island,"103,000","103,000","103,000","104,000","104,000","104,000","105,000","106,000","107,000","107,000","107,000","107,000","107,000","108,000","108,000","109,000","109,000","109,000","109,000","109,000","109,000","109,000","109,000","109,000","108,000","108,000","109,000","109,000","109,000","109,000","109,000","109,000","110,000","110,000","110,000","111,000","111,000","111,000","111,000","111,000",...,"141,654","142,732","143,020","143,265","143,963","144,378","144,310","144,472","144,530","144,520","144,175","143,948","144,094","144,103","143,996","143,984","144,283","144,538","144,368","144,259","144,546","144,894","145,202","145,784","146,969","147,963","148,228","148,921","150,483","151,478","151,648","152,208","153,584","155,085","155,111","155,656","156,947","157,901","158,158","158,717"
Nova Scotia,"722,000","725,000","728,000","731,000","732,000","735,000","738,000","740,000","742,000","744,000","746,000","748,000","747,000","749,000","751,000","752,000","752,000","753,000","755,000","756,000","755,000","754,000","756,000","755,000","754,000","755,000","757,000","756,000","757,000","758,000","761,000","762,000","764,000","765,000","768,000","771,000","771,000","774,000","777,000","778,000",...,"942,107","944,179","944,068","943,918","944,274","945,016","944,307","943,746","943,635","943,341","942,598","941,230","940,434","940,105","9

In [ ]:
canadian_pop_estimates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, Canada to Nunavut 5
Columns: 242 entries, Q1 1960 to Q2 2020
dtypes: object(242)
memory usage: 26.6+ KB


In [ ]:
# select only data for Canada
Population=canadian_pop_estimates.loc['Canada']

#check class/type
print(type(Population))

<class 'pandas.core.series.Series'>


In [ ]:
Population

Q1 1960    17,710,000
Q2 1960    17,793,000
Q3 1960    17,909,000
Q4 1960    18,009,000
Q1 1961    18,092,000
              ...    
Q2 2019    37,408,205
Q3 2019    37,589,262
Q4 2019    37,797,496
Q1 2020    37,894,799
Q2 2020    37,971,020
Name: Canada, Length: 242, dtype: object

In [ ]:
Pop = Population.str.replace(',','', regex=True)

In [ ]:
# convert series to dataframe

Pop_year= pd.DataFrame(Pop)
Pop_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 242 entries, Q1 1960 to Q2 2020
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Canada  242 non-null    object
dtypes: object(1)
memory usage: 13.8+ KB


In [ ]:
## inspect 
Pop_year.head()

,Canada
Q1 1960,17710000
Q2 1960,17793000
Q3 1960,17909000
Q4 1960,18009000
Q1 1961,18092000


In [ ]:
Pop_year.reset_index(inplace=True)

In [ ]:
# rename columns 
Pop_year.rename(columns={'index': 'Quarter_Year', 'Canada': 'Population Estimates'}, inplace=True)

In [ ]:
Pop_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Quarter_Year          242 non-null    object
 1   Population Estimates  242 non-null    object
dtypes: object(2)
memory usage: 3.9+ KB


In [ ]:
# change dtypes to string and integer

Pop_year.loc[:,'Quarter_Year'] = Pop_year.loc[:,'Quarter_Year'].astype('string')
Pop_year['Population Estimates'] = Pop_year['Population Estimates'].astype('int64')

Pop_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Quarter_Year          242 non-null    string
 1   Population Estimates  242 non-null    int64 
dtypes: int64(1), string(1)
memory usage: 3.9 KB


In [ ]:
Pop_year.head()

,Quarter_Year,Population Estimates
0,Q1 1960,17710000
1,Q2 1960,17793000
2,Q3 1960,17909000
3,Q4 1960,18009000
4,Q1 1961,18092000


In [ ]:
## add columns for Quarter and Year by splitting Quarter_Year Column

splitframe= Pop_year['Quarter_Year'].str.split(' ', n=1, expand=True)

Pop_year['Quarter'] = splitframe[0]
Pop_year['Year'] = splitframe[1].astype('int64')

In [ ]:
Pop_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Quarter_Year          242 non-null    string
 1   Population Estimates  242 non-null    int64 
 2   Quarter               242 non-null    string
 3   Year                  242 non-null    int64 
dtypes: int64(2), string(2)
memory usage: 7.7 KB


In [ ]:
Pop_year.head()

,Quarter_Year,Population Estimates,Quarter,Year
0,Q1 1960,17710000,Q1,1960
1,Q2 1960,17793000,Q2,1960
2,Q3 1960,17909000,Q3,1960
3,Q4 1960,18009000,Q4,1960
4,Q1 1961,18092000,Q1,1961


In [ ]:
# drop all rows where Quarter is not Q1

Pop_year_Q1 = Pop_year.loc[Pop_year['Quarter'] == 'Q1']

Pop_year_Q1.head()

,Quarter_Year,Population Estimates,Quarter,Year
0,Q1 1960,17710000,Q1,1960
4,Q1 1961,18092000,Q1,1961
8,Q1 1962,18442000,Q1,1962
12,Q1 1963,18787000,Q1,1963
16,Q1 1964,19142000,Q1,1964


In [ ]:
# drop quarter_year, and quarter columns
Pop_year_df = Pop_year_Q1.drop(columns=['Quarter_Year','Quarter'], inplace=False).set_index('Year')

Pop_year_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 1960 to 2020
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Population Estimates  61 non-null     int64
dtypes: int64(1)
memory usage: 976.0 bytes


In [ ]:
#Pop_year_df.columns = Pop_year_df.columns.get_level_values(0)

# Transform Debt Datasets for Net Debt 

## Start with the Archived Datasets from 1960 to 2008

In [ ]:
canadian_fed_debt_archived.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
Government debt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Gross federal government debt,"20,399","20,864","22,775","24,494","26,212","26,838","27,738","29,844","32,008","34,355","35,820","39,930","43,762","46,211","49,057","55,125","61,928","69,735","82,355","100,492","110,611","127,665","144,505","173,091","209,304","250,515","284,037","318,277","349,894","379,993","406,606","444,557","476,104","514,357","557,604","595,877","634,939","651,124","645,725","648,389","648,212","644,900","640,526","629,638","628,830","626,217","619,701","618,765","594,390"
Unmatured debt,"15,890","16,068","16,946","17,962","18,740","18,978","19,110","19,940","20,580","22,101","22,637","25,201","27,258","29,038","29,171","33,086","37,697","42,519","51,568","66,591","72,121","82,649","92,353","115,476","142,126","172,797","201,517","229,492","252,058","277,624","295,985","325,212","352,905","383,798","414,942","441,991","470,581","477,940","468,024","461,004","457,331","447,741","444,058","441,366","437,946","432,996","428,354","419,647","394,734"
Marketable bonds,"10,628","10,578","11,006","11,215","11,418","11,286","11,227","11,612","11,998","13,082","13,148","13,640","13,689","13,725","13,818","14,490","15,623","17,888","21,328","29,473","35,890","43,724","46,724","51,714","58,994","71,373","86,957","100,294","110,222","121,121","131,810","147,104","161,499","181,322","208,464","233,621","262,279","295,022","309,256","315,421","315,854","316,651","314,685","303,689","292,145","276,676","269,577","264,664","259,998"
Treasury bills,"2,125","1,935","1,885","2,165","2,230","2,140","2,150","2,310","2,480","2,840","2,895","3,735","3,830","4,290","4,905","5,630","6,495","8,255","11,295","13,535","16,325","21,770","19,375","29,125","41,700","52,300","61,950","76,950","81,050","102,700","118,550","139,150","152,300","162,050","166,000","164,450","166,100","135,400","112,300","96,950","99,850","88,700","94,201","104,600","113,400","127,200","131,600","134,100","117,000"
Notes and loans,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,850,"4,240","1,712","1,207","1,122","1,825","3,039","6,960","7,921","6,142","4,971","3,042","1,623","1,022",7,"2,552","5,649","9,046","7,296","10,557","12,533","16,353","11,302","12,570","7,765","7,124","7,720","6,705","6,740","3,966","3,626"


In [ ]:
archived_fed_debt = canadian_fed_debt_archived.loc[['Net federal government financial debt', 'Gross federal government debt']]

In [ ]:
# check class/type
print(type(archived_fed_debt))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
# remove commas to be able to change to numeric later
fed_debt_archived = archived_fed_debt.replace(',','', regex=True)
fed_debt_archived.head()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008
Government debt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Net federal government financial debt,12047,12394,13378,14079,15262,15748,15381,15866,16713,17396,18095,18581,19328,20123,21580,24769,28573,32629,45846,59040,72555,86280,99600,128302,164532,209891,245151,276735,305438,333519,362920,395075,428682,471061,513219,550685,578718,588402,581581,574468,561733,545300,534690,526492,523648,523344,514099,508122,490412
Gross federal government debt,20399,20864,22775,24494,26212,26838,27738,29844,32008,34355,35820,39930,43762,46211,49057,55125,61928,69735,82355,100492,110611,127665,144505,173091,209304,250515,284037,318277,349894,379993,406606,444557,476104,514357,557604,595877,634939,651124,645725,648389,648212,644900,640526,629638,628830,626217,619701,618765,594390


In [ ]:
# create series for each government debt type
fed_net_debt_arc = fed_debt_archived.loc['Net federal government financial debt'].astype('float')
fed_gross_debt_arc = fed_debt_archived.loc['Gross federal government debt'].astype('float')

In [ ]:
# confirm series
print(type(fed_net_debt_arc))
print(type(fed_gross_debt_arc))


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [ ]:
# convert series to dataframe

fed_debt_arc_year= pd.concat([fed_net_debt_arc, fed_gross_debt_arc], axis=1).reset_index()
fed_debt_arc_year.info()

,index,Net federal government financial debt,Gross federal government debt
0,1960,12047.0,20399.0
1,1961,12394.0,20864.0
2,1962,13378.0,22775.0
3,1963,14079.0,24494.0
4,1964,15262.0,26212.0


In [ ]:
# rename columns
fed_debt_arc_year.rename(columns={'Net federal government financial debt':'Net federal government financial debt (million dollars)', 'Gross federal government debt':'Gross federal government debt (million dollars)', 'index': 'Year'}, inplace=True)

fed_debt_arc_year['Year'] = fed_debt_arc_year['Year'].astype('int64')

fed_debt_arc_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Year                                                     49 non-null     int64  
 1   Net federal government financial debt (million dollars)  49 non-null     float64
 2   Gross federal government debt (million dollars)          49 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.3 KB


In [ ]:
fed_debt_arc_year.set_index('Year', inplace=True)
fed_debt_arc_year.head()

,Net federal government financial debt (million dollars),Gross federal government debt (million dollars)
Year,,
1960,12047.0,20399.0
1961,12394.0,20864.0
1962,13378.0,22775.0
1963,14079.0,24494.0
1964,15262.0,26212.0


## Next dataset from 2009 to 2020 
### Assumed Federal Debt based on Fiscal Year End as of March every year. Data not provided for 2009, assumed value for Sept 2009 for that year only. 

In [ ]:
canadian_fed_debt_new.head()

,REF_DATE,GEO,DGUID,Central government debt,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2009-04-01,Canada,2016A000011124,"A. Federal debt (accumulated deficit), (B - E)",Dollars,81,millions,6,v86822802,1.1,NaN,..,NaN,NaN,0
1,2009-04-01,Canada,2016A000011124,"B. Net debt, (C - D)",Dollars,81,millions,6,v86822803,1.2,NaN,..,NaN,NaN,0
2,2009-04-01,Canada,2016A000011124,"C. Liabilities, gross debt",Dollars,81,millions,6,v86822804,1.3,NaN,..,NaN,NaN,0
3,2009-04-01,Canada,2016A000011124,Accounts payable and accrued liabilities,Dollars,81,millions,6,v86822805,1.4,NaN,..,NaN,NaN,0
4,2009-04-01,Canada,2016A000011124,Interest-bearing debt,Dollars,81,millions,6,v86822806,1.5,NaN,..,NaN,NaN,0


In [ ]:
canadian_fed_debt_new.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3886 entries, 0 to 3885
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   REF_DATE                 3886 non-null   datetime64[ns]
 1   GEO                      3886 non-null   object        
 2   DGUID                    3886 non-null   object        
 3   Central government debt  3886 non-null   object        
 4   UOM                      3886 non-null   object        
 5   UOM_ID                   3886 non-null   int64         
 6   SCALAR_FACTOR            3886 non-null   object        
 7   SCALAR_ID                3886 non-null   int64         
 8   VECTOR                   3886 non-null   object        
 9   COORDINATE               3886 non-null   float64       
 10  VALUE                    2388 non-null   float64       
 11  STATUS                   1498 non-null   object        
 12  SYMBOL                   0 non-nul

In [ ]:
fed_debt = canadian_fed_debt_new.loc[(canadian_fed_debt_new['Central government debt'] =='B. Net debt, (C - D)') | (canadian_fed_debt_new ['Central government debt'] == 'C. Liabilities, gross debt')]

fed_debt.head(12)

,REF_DATE,GEO,DGUID,Central government debt,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
1,2009-04-01,Canada,2016A000011124,"B. Net debt, (C - D)",Dollars,81,millions,6,v86822803,1.2,NaN,..,NaN,NaN,0
2,2009-04-01,Canada,2016A000011124,"C. Liabilities, gross debt",Dollars,81,millions,6,v86822804,1.3,NaN,..,NaN,NaN,0
30,2009-05-01,Canada,2016A000011124,"B. Net debt, (C - D)",Dollars,81,millions,6,v86822803,1.2,NaN,..,NaN,NaN,0
31,2009-05-01,Canada,2016A000011124,"C. Liabilities, gross debt",Dollars,81,millions,6,v86822804,1.3,NaN,..,NaN,NaN,0
59,2009-06-01,Canada,2016A000011124,"B. Net debt, (C - D)",Dollars,81,millions,6,v86822803,1.2,NaN,..,NaN,NaN,0
60,2009-06-01,Canada,2016A000011124,"C. Liabilities, gross debt",Dollars,81,millions,6,v86822804,1.3,NaN,..,NaN,NaN,0
88,2009-07-01,Canada,2016A000011124,"B. Net debt, (C - D)",Dollars,81,millions,6,v86822803,1.2,NaN,..,NaN,NaN,0
89,2009-07-01,Canada,2016A000011124,"C. Liabilities, gross debt",Dollars,81,millions,6,v86822804,1.3,NaN,..,NaN,NaN,0
117,2009-08-01,Canada,2016A000011124,"B. Net debt, (C - D)",Dollars,81,millions,6,v86822803,1.2,NaN,..,NaN,NaN,0
118,2009-08-01,Canada,2016A000011124,"C. Liabilities, gross debt",Dollars,81,millions,6,v86822804,1.3,NaN,..,NaN,NaN,0


In [ ]:
# select columns wanted
fed_debt = fed_debt.loc[:, ['REF_DATE', 'GEO', 'Central government debt', 'VALUE']]
fed_debt.head()


,REF_DATE,GEO,Central government debt,VALUE
1,2009-04-01,Canada,"B. Net debt, (C - D)",NaN
2,2009-04-01,Canada,"C. Liabilities, gross debt",NaN
30,2009-05-01,Canada,"B. Net debt, (C - D)",NaN
31,2009-05-01,Canada,"C. Liabilities, gross debt",NaN
59,2009-06-01,Canada,"B. Net debt, (C - D)",NaN


In [ ]:
# add Year and Month Columns

fed_debt['Year'] = fed_debt['REF_DATE'].dt.year
fed_debt['Month'] = fed_debt['REF_DATE'].dt.month

fed_debt.head()

,REF_DATE,GEO,Central government debt,VALUE,Year,Month
1,2009-04-01,Canada,"B. Net debt, (C - D)",NaN,2009,4
2,2009-04-01,Canada,"C. Liabilities, gross debt",NaN,2009,4
30,2009-05-01,Canada,"B. Net debt, (C - D)",NaN,2009,5
31,2009-05-01,Canada,"C. Liabilities, gross debt",NaN,2009,5
59,2009-06-01,Canada,"B. Net debt, (C - D)",NaN,2009,6


In [ ]:
# create dataframe for 2009 data not provided for March 2009 so using Sept 2009 as closest estimate
fed_debt_2009 = fed_debt.loc[(fed_debt['Year'] == 2009) & (fed_debt['Month']==9)]

# create dataframe for 2010 to 2020
fed_debt_2010_onwards = fed_debt.loc[fed_debt['Month']==3]

In [ ]:
# append both dataframes and drop all Columns except for Year and Value
fed_debt_new = fed_debt_2009.append(fed_debt_2010_onwards).drop(columns=['REF_DATE', 'Month', 'GEO'])

fed_debt_new.head()

,Central government debt,VALUE,Year
146,"B. Net debt, (C - D)",553446.0,2009
147,"C. Liabilities, gross debt",836957.0,2009
320,"B. Net debt, (C - D)",574715.0,2010
321,"C. Liabilities, gross debt",876083.0,2010
668,"B. Net debt, (C - D)",616463.0,2011


In [ ]:
# create separate dataframes for net debt and gross debt 

fed_net_debt_new = fed_debt_new.loc[fed_debt_new['Central government debt']=='B. Net debt, (C - D)'].drop(columns='Central government debt').rename(columns={'VALUE':'Net federal government financial debt (million dollars)'})
fed_gross_debt_new = fed_debt_new.loc[fed_debt_new['Central government debt']=='C. Liabilities, gross debt'].drop(columns='Central government debt').rename(columns={'VALUE':'Gross federal government debt (million dollars)'})

# check type/class

print(type(fed_net_debt_new))
print(type(fed_gross_debt_new))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# merge the new dataframes 

fed_debt_2009andup = fed_net_debt_new.merge(fed_gross_debt_new, how='outer', on=['Year'], validate='1:1' ).set_index('Year')

fed_debt_2009andup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 2009 to 2020
Data columns (total 2 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Net federal government financial debt (million dollars)  12 non-null     float64
 1   Gross federal government debt (million dollars)          12 non-null     float64
dtypes: float64(2)
memory usage: 288.0 bytes


## Append archived data to latest data for federal debt

In [ ]:
fed_debt_year_df = fed_debt_arc_year.append(fed_debt_2009andup, verify_integrity=True)

fed_debt_year_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 1960 to 2020
Data columns (total 2 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Net federal government financial debt (million dollars)  61 non-null     float64
 1   Gross federal government debt (million dollars)          61 non-null     float64
dtypes: float64(2)
memory usage: 3.9 KB


# Merge all datasets to create one master dataframe 

In [ ]:
fed_debt_year_df
Pop_year_df
GDP_year_df

merged_df = fed_debt_year_df.join(Pop_year_df)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 1960 to 2020
Data columns (total 3 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Net federal government financial debt (million dollars)  61 non-null     float64
 1   Gross federal government debt (million dollars)          61 non-null     float64
 2   Population Estimates                                     61 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 4.4 KB


In [ ]:
# merge GDP dataset 

merged_df_final = merged_df.join(GDP_year_df, how='right')

merged_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1961 to 2020
Data columns (total 4 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Net federal government financial debt (million dollars)    60 non-null     float64
 1   Gross federal government debt (million dollars)            60 non-null     float64
 2   Population Estimates                                       60 non-null     int64  
 3   Gross domestic product at market prices (million dollars)  60 non-null     int64  
dtypes: float64(2), int64(2)
memory usage: 4.8 KB


In [ ]:
display(merged_df_final)

,Net federal government financial debt (million dollars),Gross federal government debt (million dollars),Population Estimates,Gross domestic product at market prices (million dollars)
Year,,,,
1961,12394.0,20864.0,18092000,40672
1962,13378.0,22775.0,18442000,44696
1963,14079.0,24494.0,18787000,47668
1964,15262.0,26212.0,19142000,52652
1965,15748.0,26838.0,19501000,57364
1966,15381.0,27738.0,19857000,64428
1967,15866.0,29844.0,20228000,69584
1968,16713.0,32008.0,20581000,75408
1969,17396.0,34355.0,20888000,83876


# Add some Calculated Columns 

In [ ]:
merged_df_final['Federal Net Debt per person (dollars)'] = merged_df_final['Net federal government financial debt (million dollars)'].div(merged_df_final['Population Estimates']).mul(1000000).round(2)
merged_df_final['Federal Gross Debt per person (dollars)'] = merged_df_final['Gross federal government debt (million dollars)'].div(merged_df_final['Population Estimates']).mul(1000000).round(2)
merged_df_final['Federal GDP per person (dollars)'] = merged_df_final['Gross domestic product at market prices (million dollars)'].div(merged_df_final['Population Estimates']).mul(1000000).round(2)
merged_df_final['Federal Net Debt as a percent of GDP (%)'] = merged_df_final['Net federal government financial debt (million dollars)'].div(merged_df_final['Gross domestic product at market prices (million dollars)']).mul(100).round(2)
merged_df_final['Federal Gross Debt as a percent of GDP (%)'] = merged_df_final['Gross federal government debt (million dollars)'].div(merged_df_final['Gross domestic product at market prices (million dollars)']).mul(100).round(2)
merged_df_final.head()

,Net federal government financial debt (million dollars),Gross federal government debt (million dollars),Population Estimates,Gross domestic product at market prices (million dollars),Federal Net Debt per person (dollars),Federal Gross Debt per person (dollars),Federal GDP per person (dollars),Federal Net Debt as a percent of GDP (%),Federal Gross Debt as a percent of GDP (%)
Year,,,,,,,,,
1961,12394.0,20864.0,18092000,40672,685.05,1153.22,2248.07,30.47,51.30
1962,13378.0,22775.0,18442000,44696,725.41,1234.95,2423.60,29.93,50.96
1963,14079.0,24494.0,18787000,47668,749.40,1303.77,2537.29,29.54,51.38
1964,15262.0,26212.0,19142000,52652,797.30,1369.34,2750.60,28.99,49.78
1965,15748.0,26838.0,19501000,57364,807.55,1376.24,2941.59,27.45,46.79


# Download merged dataframe

In [ ]:
merged_df_final.to_csv("Federal Debt, Population and GDP.csv", sep=",", index=True)
files.download("Federal Debt, Population and GDP.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>